## Im2Col + GEMM

In [23]:
import os
import random

import numpy as np
import torch

# from torchvision.io import decode_image
# from torchvision.models import ResNet18_Weights, resnet18
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# Set the global seed for NumPy's random number generator
seed_value = 42
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


def set_seed(seed: int = 42) -> None:
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # Set a fixed value for the hash seed
    os.environ["PYTHONHASHSEED"] = str(seed)
    print(f"Random seed set as {seed}")


set_seed(42)

Random seed set as 42


In [ ]:
from classify_single import classify_single_image, evaluate_model
from conv2dgemm import Conv2dGEMM

In [25]:
class LeNet(nn.Module):
    def __init__(self, num_classes=10, kernel_size=5):
        super(LeNet, self).__init__()
        self.conv1 = Conv2dGEMM(in_channels=3, out_channels=6, kernel_size=kernel_size)
        self.conv2 = Conv2dGEMM(in_channels=6, out_channels=16, kernel_size=kernel_size)

        self.pool = nn.MaxPool2d(2, 2)

        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        # x = x.view(x.shape[0], -1)
        x = x.reshape(x.shape[0], -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [ ]:
def weights_init(m):
    if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
        nn.init.kaiming_normal_(m.weight, mode="fan_out", nonlinearity="relu")
        if m.bias is not None:
            nn.init.constant_(m.bias, 0)


model = LeNet(10)
# model.reset_params()
model.apply(weights_init)

model.to(device)

LeNet(
  (conv1): Conv2dGEMM()
  (conv2): Conv2dGEMM()
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [30]:
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
)


train_dataset = datasets.CIFAR10(
    root="./data", train=True, download=True, transform=transform
)
test_dataset = datasets.CIFAR10(
    root="./data", train=False, download=True, transform=transform
)

class_names = [
    "airplane",
    "automobile",
    "bird",
    "cat",
    "deer",
    "dog",
    "frog",
    "horse",
    "ship",
    "truck",
]

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [31]:
criterion = nn.CrossEntropyLoss()
# optimizer = optim.Adam(model.parameters(), lr=0.01)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
num_epochs = 10

In [ ]:
import time

model.train()

for epoch in range(num_epochs):
    running_loss = 0.0
    correct = 0
    total = 0

    total_forward_time = 0.0
    total_backward_time = 0.0
    total_forward_memory = 0.0
    total_backward_memory = 0.0

    for batch_idx, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        torch.cuda.reset_peak_memory_stats()

        # ------- FORWARD -------
        torch.cuda.synchronize()
        mem_before_forward = torch.cuda.memory_allocated()

        forward_start = time.perf_counter()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        torch.cuda.synchronize()
        forward_end = time.perf_counter()

        forward_time = forward_end - forward_start
        total_forward_time += forward_time

        mem_after_forward = torch.cuda.memory_allocated()
        forward_memory = (mem_after_forward - mem_before_forward) / 1024**2
        total_forward_memory += forward_memory

        # ------- BACKWARD -------
        torch.cuda.synchronize()

        # reset peak memory stats for isolated backward measurement
        torch.cuda.reset_peak_memory_stats()

        backward_start = time.perf_counter()
        loss.backward()
        torch.cuda.synchronize()
        backward_end = time.perf_counter()

        backward_time = backward_end - backward_start
        total_backward_time += backward_time

        # peak memory used inside backward
        backward_peak = torch.cuda.max_memory_allocated() / 1024**2
        total_backward_memory += backward_peak

        optimizer.step()

        # stats
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    # epoch metrics
    avg_forward = total_forward_time / len(train_loader)
    avg_backward = total_backward_time / len(train_loader)
    avg_forward_mem = total_forward_memory / len(train_loader)
    avg_backward_mem = total_backward_memory / len(train_loader)
    peak_memory = torch.cuda.max_memory_allocated() / 1024**2

    print(
        f"Epoch [{epoch + 1}/{num_epochs}] "
        f"Loss: {running_loss / len(train_loader):.4f}, "
        f"Acc: {100 * correct / total:.2f}%, "
        f"Fwd: {avg_forward * 1000:.2f} ms ({avg_forward_mem:.2f} MB), "
        f"Bwd: {avg_backward * 1000:.2f} ms ({avg_backward_mem:.2f} MB), "
        f"Peak: {peak_memory:.2f} MB"
    )


Epoch [1/10] Loss: 1.7730, Acc: 34.70%, Fwd: 3.90 ms (21.51 MB), Bwd: 1.62 ms (62.54 MB), Peak: 41.20 MB
Epoch [2/10] Loss: 1.5164, Acc: 44.51%, Fwd: 3.47 ms (21.50 MB), Bwd: 1.42 ms (62.39 MB), Peak: 41.20 MB
Epoch [3/10] Loss: 1.4357, Acc: 47.96%, Fwd: 3.47 ms (21.50 MB), Bwd: 1.43 ms (62.39 MB), Peak: 41.20 MB
Epoch [4/10] Loss: 1.3740, Acc: 50.59%, Fwd: 3.47 ms (21.50 MB), Bwd: 1.45 ms (62.39 MB), Peak: 41.20 MB
Epoch [5/10] Loss: 1.3300, Acc: 52.27%, Fwd: 3.47 ms (21.50 MB), Bwd: 1.41 ms (62.39 MB), Peak: 41.20 MB
Epoch [6/10] Loss: 1.2957, Acc: 53.70%, Fwd: 3.47 ms (21.50 MB), Bwd: 1.40 ms (62.39 MB), Peak: 41.20 MB
Epoch [7/10] Loss: 1.2508, Acc: 55.26%, Fwd: 3.47 ms (21.50 MB), Bwd: 1.43 ms (62.39 MB), Peak: 41.20 MB
Epoch [8/10] Loss: 1.2316, Acc: 55.77%, Fwd: 3.46 ms (21.50 MB), Bwd: 1.41 ms (62.39 MB), Peak: 41.20 MB
Epoch [9/10] Loss: 1.2015, Acc: 56.92%, Fwd: 3.47 ms (21.50 MB), Bwd: 1.41 ms (62.39 MB), Peak: 41.20 MB
Epoch [10/10] Loss: 1.1713, Acc: 58.22%, Fwd: 3.61 ms (

In [ ]:
import json

acc = evaluate_model(model, test_loader, device)
report = {
    "name": "lenet",
    "batch_size": 64,
    "avg_forward": avg_forward,
    "avg_backward": avg_backward,
    "avg_forward_mem": avg_forward_mem,
    "avg_backward_mem": avg_backward_mem,
    "kernel_size": 5,
    "acc": acc,
}

file_path = "lenetgemm_kernel5_bs64.json"

with open(file_path, "w") as f:
    json.dump(report, f, indent=4)  # indent=4 makes the JSON file human-readable

Test Accuracy: 53.16%


In [ ]:
from PIL import Image

transform = transforms.Compose(
    [
        transforms.Resize((32, 32)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ]
)

image = Image.open("/content/cat_watermelon.jpg").convert("RGB")
sample_image = transform(image).unsqueeze(0).to(device)


In [ ]:
from torch.profiler import ProfilerActivity, profile


def on_trace_ready(prof):
    prof.export_chrome_trace("trace.json")


with profile(
    activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA],
    on_trace_ready=on_trace_ready,
) as prof:
    # batch = preprocess(img).unsqueeze(0)

    # # Step 4: Use the model and print the predicted category
    # prediction = model(batch).squeeze(0).softmax(0)
    # class_id = prediction.argmax().item()
    # score = prediction[class_id].item()
    # category_name = weights.meta["categories"][class_id]
    classify_single_image(model, sample_image, device, class_names)

In [36]:
print(prof.key_averages().table(sort_by="cuda_time_total", row_limit=10))


-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                           aten::linear         0.08%      21.098us        63.76%      17.299ms       5.766ms       0.000us         0.00%      50.398us      16.799us             3  
                                            aten::addmm        14.83%       4.022ms        63.59%      17.253ms       5.751ms      19.743us        10.44%      50.398us      16.799us             3  
         